In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import preprocessing
import re


In [17]:
path_treino = 'data\cars_train.csv'
path_test = 'data\cars_test.csv'
dados_treino = pd.read_csv(path_treino)
dados_treino = dados_treino.drop(columns=['id', 'num_fotos','elegivel_revisao','dono_aceita_troca','entrega_delivery','cidade_vendedor'])

#transformando a colunas que tem NaN e informaçoes descritivas no modelo verdadeiro e falso como garatia de fabriga para sim e não
dados_treino.loc[dados_treino['garantia_de_fábrica'].notnull(),'garantia_de_fábrica'] = 'S'
dados_treino.loc[dados_treino['garantia_de_fábrica'].isnull(),'garantia_de_fábrica'] = 'N'

#transformando os dados veiculo unico dono
dados_treino.loc[dados_treino['veiculo_único_dono'].notnull(),'veiculo_único_dono'] = 'S'
dados_treino.loc[dados_treino['veiculo_único_dono'].isnull(),'veiculo_único_dono'] = 'N'

#ipva_pago
dados_treino.loc[dados_treino['ipva_pago'].notnull(),'ipva_pago'] = 'S'
dados_treino.loc[dados_treino['ipva_pago'].isnull(),'ipva_pago'] = 'N'

#veiculo_licenciado	
dados_treino.loc[dados_treino['veiculo_licenciado'].notnull(),'veiculo_licenciado'] = 'S'
dados_treino.loc[dados_treino['veiculo_licenciado'].isnull(),'veiculo_licenciado'] = 'N'

#garantia_de_fábrica
dados_treino.loc[dados_treino['garantia_de_fábrica'].notnull(),'garantia_de_fábrica'] = 'S'
dados_treino.loc[dados_treino['garantia_de_fábrica'].isnull(),'garantia_de_fábrica'] = 'N'

#revisoes_dentro_agenda
dados_treino.loc[dados_treino['revisoes_dentro_agenda'].notnull(),'revisoes_dentro_agenda'] = 'S'
dados_treino.loc[dados_treino['revisoes_dentro_agenda'].isnull(),'revisoes_dentro_agenda'] = 'N'

#revisoes_concessionaria
dados_treino.loc[dados_treino['revisoes_concessionaria'].notnull(),'revisoes_concessionaria'] = 'S'
dados_treino.loc[dados_treino['revisoes_concessionaria'].isnull(),'revisoes_concessionaria'] = 'N'

#veiculo_alienado
dados_treino.loc[dados_treino['veiculo_alienado'].notnull(),'veiculo_alienado'] = 'S'
dados_treino.loc[dados_treino['veiculo_alienado'].isnull(),'veiculo_alienado'] = 'N'


In [18]:
dados_treino['ano_atual'] = 2023
dados_treino['anos_de_fabricado'] = dados_treino['ano_atual'] - dados_treino['ano_de_fabricacao']
dados_treino.drop(['ano_atual'], inplace = True, axis = 1)
dados_treino.drop(['ano_de_fabricacao'], inplace = True, axis = 1)

dados_treino

,marca,modelo,versao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,...,troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco,anos_de_fabricado
0,NISSAN,KICKS,1.6 16V FLEXSTART SL 4P XTRONIC,2017,67772,CVT,4,Sedã,N,Branco,...,False,N,S,S,S,S,N,N,74732.59008,6
1,JEEP,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2017,62979,Automática,4,Sedã,N,Branco,...,False,N,N,S,N,S,N,N,81965.33263,6
2,KIA,SORENTO,2.4 16V GASOLINA EX 7L AWD AUTOMÁTICO,2019,44070,Automática,4,Sedã,N,Preto,...,False,N,N,N,N,S,N,N,162824.81450,5
3,VOLKSWAGEN,AMAROK,2.0 HIGHLINE 4X4 CD 16V TURBO INTERCOOLER DIES...,2015,85357,Automática,4,Picape,N,Branco,...,True,N,N,S,S,S,N,N,123681.35890,10
4,SSANGYONG,KORANDO,2.0 GLS 4X4 16V TURBO DIESEL 4P AUTOMÁTICO,2015,71491,Automática,4,Utilitário esportivo,N,Preto,...,False,N,S,N,N,S,S,N,82419.76389,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29579,TOYOTA,HILUX,2.8 D-4D TURBO DIESEL CD SRX 4X4 AUTOMÁTICO,2021,8150,Automática,4,Picape,N,Branco,...,False,S,N,S,S,S,N,N,403015.28960,2
29580,VOLKSWAGEN,POLO,1.0 200 TSI HIGHLINE AUTOMÁTICO,2020,17987,Automática,4,Picape,N,Branco,...,False,S,S,S,S,S,N,N,88978.08050,3
29581,TOYOTA,HILUX,2.8 SRX 4X4 CD 16V DIESEL 4P AUTOMÁTICO,2019,44742,Automática,4,Picape,N,Cinza,...,False,N,N,N,N,S,N,N,218807.64870,4
29582,PEUGEOT,2008,1.6 16V FLEX ALLURE PACK 4P AUTOMÁTICO,2022,35376,Automática,4,Sedã,N,Branco,...,False,S,N,S,S,S,N,N,68495.99069,2


Não há nenhum carro apenas a alcool
Vamos extrair informaçoes da coluna versão e expandilas em colunas 4x4, valvuvlas, cilindradas e combustível 

In [78]:
s = '2.8 D-4D TURBO DIESEL CD SRX 4X4 AUTOMÁTICO'

reg_cilindrada = r'([0-9]\.[0-9])'
reg_4x4= r'([0-9]X[0-9])'

num = re.findall(reg_4x4,s) 

print(num)

['4X4']


In [105]:
reg_cilindrada = r'([0-9]\.[0-9])'
reg_4x4= r'([0-9]X[0-9])'

dados_treino['cilindrada'] = dados_treino['versao'].str.extract(reg_cilindrada)
dados_treino['4X4'] = dados_treino['versao'].str.count(reg_4x4)
dados_treino['combutivel'] = dados_treino['versao'].str.extract(r'(FLEX|GASOLINA|DIESEL)')
dados_treino['valvulas'] = dados_treino['versao'].str.extract(r'(\d+V)')

dados_treino

,marca,modelo,versao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,...,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco,anos_de_fabricado,cilindrada,4X4,combutivel,valvulas
0,NISSAN,KICKS,1.6 16V FLEXSTART SL 4P XTRONIC,2017,67772,CVT,4,Sedã,N,Branco,...,S,S,N,N,74732.59008,6,1.6,0,FLEX,16V
1,JEEP,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2017,62979,Automática,4,Sedã,N,Branco,...,N,S,N,N,81965.33263,6,2.0,0,FLEX,16V
2,KIA,SORENTO,2.4 16V GASOLINA EX 7L AWD AUTOMÁTICO,2019,44070,Automática,4,Sedã,N,Preto,...,N,S,N,N,162824.81450,5,2.4,0,GASOLINA,16V
3,VOLKSWAGEN,AMAROK,2.0 HIGHLINE 4X4 CD 16V TURBO INTERCOOLER DIES...,2015,85357,Automática,4,Picape,N,Branco,...,S,S,N,N,123681.35890,10,2.0,1,DIESEL,16V
4,SSANGYONG,KORANDO,2.0 GLS 4X4 16V TURBO DIESEL 4P AUTOMÁTICO,2015,71491,Automática,4,Utilitário esportivo,N,Preto,...,N,S,S,N,82419.76389,10,2.0,1,DIESEL,16V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29579,TOYOTA,HILUX,2.8 D-4D TURBO DIESEL CD SRX 4X4 AUTOMÁTICO,2021,8150,Automática,4,Picape,N,Branco,...,S,S,N,N,403015.28960,2,2.8,1,DIESEL,NaN
29580,VOLKSWAGEN,POLO,1.0 200 TSI HIGHLINE AUTOMÁTICO,2020,17987,Automática,4,Picape,N,Branco,...,S,S,N,N,88978.08050,3,1.0,0,NaN,NaN
29581,TOYOTA,HILUX,2.8 SRX 4X4 CD 16V DIESEL 4P AUTOMÁTICO,2019,44742,Automática,4,Picape,N,Cinza,...,N,S,N,N,218807.64870,4,2.8,1,DIESEL,16V
29582,PEUGEOT,2008,1.6 16V FLEX ALLURE PACK 4P AUTOMÁTICO,2022,35376,Automática,4,Sedã,N,Branco,...,S,S,N,N,68495.99069,2,1.6,0,FLEX,16V
